In [1]:
import pandas as pd

In [5]:
kws = "['ELSE', 'LIMIT', 'CASE', 'SELECT', 'OR', 'FROM', 'GROUP BY', 'OFFSET', 'ORDER BY', 'CREATE', 'INNER JOIN', 'IN', 'HAVING', 'DISTINCT', 'NULL', 'ON', 'JOIN', 'WHERE', 'TABLE', 'LEFT JOIN', 'THEN', 'RIGHT JOIN', 'AND', 'LIKE', 'END', 'BETWEEN', 'NOT', 'AS', 'IS', 'WHEN']"

kws = [i.strip().replace("'", "") for i in kws[1:-1].split(',')]




['ELSE',
 'LIMIT',
 'CASE',
 'SELECT',
 'OR',
 'FROM',
 'GROUP BY',
 'OFFSET',
 'ORDER BY',
 'CREATE',
 'INNER JOIN',
 'IN',
 'HAVING',
 'DISTINCT',
 'NULL',
 'ON',
 'JOIN',
 'WHERE',
 'TABLE',
 'LEFT JOIN',
 'THEN',
 'RIGHT JOIN',
 'AND',
 'LIKE',
 'END',
 'BETWEEN',
 'NOT',
 'AS',
 'IS',
 'WHEN']